In [92]:
import torch
import itertools

def generate_instance(n, device='cpu'):
    """Generate assortment instances"""
    beta_price = torch.rand(n, device=device) * -1
    alpha = torch.rand(n, device=device)
    r_vec = torch.exp(torch.randn(n, device=device))
    w_vec = torch.exp(alpha + beta_price * r_vec)

    return r_vec, w_vec


def get_initial_order(w_vec, r_vec, C):
    """
    Select the initial order of products based on their weighted revenue.

    Parameters:
    w_vec (torch.Tensor): A tensor containing the weight vector for the products.
    r_vec (torch.Tensor): A tensor containing the revenue vector for the products.
    n (int): The number of products.
    C (int): The number of products to select.

    Returns:
    torch.Tensor: A tensor of selected product indices.
    """

    values = w_vec * r_vec
    _, sorted_indices = torch.sort(values, descending=True)
    initial_order_indices = sorted_indices[:C]

    return initial_order_indices


In [72]:
def get_z(w_vec, r_vec):
    """
    Calculate the intersection points of revenue functions for a set of products.

    Parameters:
    w_vec (torch.Tensor): A tensor containing the weight vector for the products.
    r_vec (torch.Tensor): A tensor containing the revenue vector for the products.

    Returns:
    torch.Tensor: A tensor of intersection points where the revenue functions of the products intersect.

    Description:
    The function identifies the intersection points of revenue functions for a set of products.
    These intersections are potential candidates for the optimal expected revenue.

    1. Initialize the number of products (`num_prods`).
    2. Create a list, `list_intersections`, starting with `0` and the maximum value of `r_vec`.
    3. Calculate the maximum revenue (`max_rev`) from `r_vec`.
    4. Use tensor operations to find the intersection points of their revenue functions.
       - Calculate the intersection points using the formula:
         intersection = (w_vec[i] * r_vec[i] - w_vec[j] * r_vec[j]) / (w_vec[i] - w_vec[j])
       - Add the intersection points to `list_intersections` if they are less than the maximum revenue (`max_rev`) and non-negative.
    5. Return the list of intersection points (`list_intersections`).
    """

    num_prods = len(w_vec)
    max_rev = torch.max(r_vec).item()
    list_intersections = torch.tensor([0, max_rev])

    # Create a meshgrid of indices
    i_indices, j_indices = torch.triu_indices(num_prods, num_prods, 1)
    # Calculate intersection points
    numerator = w_vec[i_indices] * r_vec[i_indices] - w_vec[j_indices] * r_vec[j_indices]
    denominator = w_vec[i_indices] - w_vec[j_indices]
    intersections = numerator / denominator
    # Filter valid intersection points
    valid_intersections = intersections[(intersections < max_rev) & (intersections >= 0)]
    list_intersections = torch.cat((list_intersections, valid_intersections))

    return list_intersections


def get_final_assort(w_vec, r_vec, C, final_z):
    """
    Select the final assortment of products based on the given criteria.

    Parameters:
    w_vec (torch.Tensor): A tensor containing the weight vector for the products.
    r_vec (torch.Tensor): A tensor containing the revenue vector for the products.
    C (int): The number of products to select.
    final_z (torch.Tensor): A tensor containing the final z value.

    Returns:
    torch.Tensor: A tensor of selected product indices.
    """

    f_vals = (r_vec - final_z) * w_vec
    _, sorted_indices = torch.sort(f_vals, descending=True)
    selected_indices = sorted_indices[:C]

    return selected_indices


def new_approach(w_vec, r_vec, C):
    """
    Our updated O(n^2) approach for selecting the final assortment of products.

    Parameters:
    w_vec (torch.Tensor): A tensor containing the weight vector for the products.
    r_vec (torch.Tensor): A tensor containing the revenue vector for the products.
    n (int): The number of products.
    C (int): The number of products to select.

    Returns:
    torch.Tensor: A tensor of selected product indices.
    """
    # Get the initial order of products
    S = get_initial_order(w_vec, r_vec, C)

    # Calculate the intersection points
    intersections = get_z(w_vec, r_vec)

    while len(intersections) > 2:
        median = torch.median(intersections).item()
        f_vals = (r_vec - median) * w_vec
        f = torch.sum(torch.topk(f_vals, C).values).item()

        if f <= median:
            intersections = intersections[intersections <= median]
        else:
            intersections = intersections[intersections >= median]

    final_z = torch.mean(intersections).item()

    # Get the final assortment of products
    S = get_final_assort(w_vec, r_vec, C, final_z)

    return S



In [96]:
def MNL_Revenue(w_vec, r_vec, S):
    """
    Computes revenue of expected revenue of assortment S.

    Parameters:
    w_vec (torch.Tensor): A tensor containing the weight vector for the products.
    r_vec (torch.Tensor): A tensor containing the revenue vector for the products.
    S (torch.Tensor): A tensor containing the indices of the selected products.

    Returns:
    float: The calculated MNL revenue for the subset S.
    """
    num = torch.sum(w_vec[S] * r_vec[S])
    denom = 1 + torch.sum(w_vec[S])
    return (num / denom).item()

def get_combinatorial_max_revenue(w_vec, r_vec, n, C):
    """
    Go through all combinatorial options and compute the maximum revenue using MNL_Revenue.

    Parameters:
    w_vec (torch.Tensor): A tensor containing the weight vector for the products.
    r_vec (torch.Tensor): A tensor containing the revenue vector for the products.
    n (int): The number of products.
    C (int): The number of products to select.

    Returns:
    torch.Tensor: A tensor of selected product indices with the maximum revenue.
    float: The maximum revenue.
    """
    max_revenue = -float('inf')
    best_subset = None

    # Generate all combinations of n choose C
    for subset in itertools.combinations(range(n), C):
        subset_tensor = torch.tensor(subset, dtype=torch.long)
        revenue = MNL_Revenue(w_vec, r_vec, subset_tensor)
        if revenue > max_revenue:
            max_revenue = revenue
            best_subset = subset_tensor

    return best_subset, max_revenue


n_list = [10]
C = 3
num_trials = 10

rev_diff = []

for n in n_list:
  for trial in range(num_trials):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    r_vec, w_vec = generate_instance(n, device=device)
    S_new = new_approach(w_vec, r_vec, C)
    S_brutalforce, max_revenue = get_combinatorial_max_revenue(w_vec, r_vec, n, C)
    if MNL_Revenue(w_vec, r_vec, S_new) != max_revenue:
      rev_diff.append(MNL_Revenue(w_vec, r_vec, S_new) - max_revenue)

print("Average difference in revenue:", sum(rev_diff) / len(rev_diff))


Average difference in revenue: -5.960464477539063e-08
